In [ ]:
!pip install transformers datasets huggingface_hub

In [ ]:
from transformers import AutoModel, AutoTokenizer
model_id = "klue/bert-base"
model = AutoModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from datasets import load_dataset

imdb_ds = load_dataset("yangwang825/klue-ynat")

In [ ]:
imdb_ds["train"][0]

{'text': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3}

In [ ]:
train_ds = imdb_ds["train"]
test_ds = imdb_ds["validation"]

In [ ]:

import torch
import numpy as np
from transformers import (
    Trainer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    AutoTokenizer
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


model_id = "klue/bert-base"
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=len(train_ds.features['label'].names)
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    learning_rate=5e-5,
    push_to_hub=False,
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

print(trainer.evaluate(test_ds))

trainer.train()

print(trainer.evaluate(test_ds))

KeyboardInterrupt: 